In [1]:
from init import *

### store_i64

```python
def store_i64(scope: Union[uint64_t, str], table: Union[uint64_t, str], payer: Union[uint64_t, str], id: Union[uint64_t, str],  data: bytes) -> int32_t
```

In [7]:
code = '''
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    itr = chain.db_find_i64(code, scope, table, id)
    chain.db_remove_i64(1)
    itr = chain.db_store_i64(scope, table, payer, id, b'world')
    print(itr)
    print(chain.db_get_i64(itr))
'''
code = uuosapi.mp_compile(python_contract, code)
uuosapi.deploy_python_code(python_contract, code, deploy_type=1)

r = uuosapi.push_action(python_contract, 'sayhello', b'hello,world', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


2
b'world'



### find_i64

```python
def find_i64(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str], id: Union[uint64_t, str]) -> int32_t
```

In [9]:
code = '''
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    itr = chain.db_find_i64(code, scope, table, id)
    print(itr)
'''
code = uuosapi.mp_compile(python_contract, code)
uuosapi.deploy_python_code(python_contract, code, deploy_type=1)

r = uuosapi.push_action(python_contract, 'sayhello', b'hello,world', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


1



### get_i64

```python
def get_i64(iterator: int32_t) -> bytes
```

In [11]:
code = '''
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'

    itr = chain.db_find_i64(code, scope, table, id)
    print(itr)
    data = chain.db_get_i64(itr)
    print(data)
'''

code = uuosapi.mp_compile(python_contract, code)
uuosapi.deploy_python_code(python_contract, code, deploy_type=1)

r = uuosapi.push_action(python_contract, 'sayhello', b'hello,world', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


1
b'world'



### update_i64

```python
def update_i64(iterator: int32_t, payer: Union[uint64_t, str], data: bytes) -> None
```

In [12]:
code = '''
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    data = chain.read_action_data()
    print(data)
    
    itr = chain.db_find_i64(code, scope, table, id)
    print(itr)
    if itr < 0:
        chain.db_store_i64(scope, table, payer, id, b'world')
    else:
        old_data = chain.db_get_i64(itr)
        print(old_data)
        chain.db_update_i64(itr, receiver, data)
'''

code = uuosapi.mp_compile(python_contract, code)
uuosapi.deploy_python_code(python_contract, code, deploy_type=1)

r = uuosapi.push_action(python_contract, 'sayhello', b'hello,world', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


b'hello,world'
1
b'world'



### lowerbound_i64

```python
def lowerbound_i64(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str], id: Union[uint64_t, str]) -> int32_t
```

### upperbound_i64

```python
def upperbound_i64(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str], id: Union[uint64_t, str]) -> int32_t
```

In [21]:
abi = ''
code = '''
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    data = chain.read_action_data()
    print(data)
    if action == name('destroy'):
        while True:
            itr = chain.db_end_i64(code, scope, table)
            if itr == -1:
                break
            itr, primary = chain.db_previous_i64(itr)
            if itr < 0:
                break
            chain.db_remove_i64(itr)
        print('destroy done!')
    elif action == name('initdb'):
        chain.db_store_i64(scope, table, payer, 1, b'1')
        chain.db_store_i64(scope, table, payer, 3, b'3')
        chain.db_store_i64(scope, table, payer, 5, b'5')
        chain.db_store_i64(scope, table, payer, 7, b'7')
        print('init done!')
    elif action == name('test'):
        itr = chain.db_upperbound_i64(code, scope, table, 3)
        data = chain.db_get_i64(itr)
        print('upperbound of 3:', data)

        itr = chain.db_upperbound_i64(code, scope, table, 2)
        data = chain.db_get_i64(itr)
        print('upperbound of 2:', data)

        itr = chain.db_lowerbound_i64(code, scope, table, 3)
        data = chain.db_get_i64(itr)
        print('lowerbound of 3:', data)

        itr = chain.db_lowerbound_i64(code, scope, table, 2)
        data = chain.db_get_i64(itr)
        print('lowerbound of 2:', data)
'''

code = uuosapi.mp_compile(python_contract, code)
uuosapi.deploy_python_code(python_contract, code, deploy_type=1)

r = uuosapi.push_action(python_contract, 'destroy', b'', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])


r = uuosapi.push_action(python_contract, 'initdb', b'hello,world', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])

None
destroy done!

b'hello,world'
init done!



In [22]:
r = uuosapi.push_action(python_contract, 'test', b'', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])

None
upperbound of 3: b'5'
upperbound of 2: b'3'
lowerbound of 3: b'3'
lowerbound of 2: b'3'




### next_i64

```python
def next_i64(iterator: int32_t) -> [int32_t, uint64_t]
```

### previous_i64

```python
def previous_i64(iterator: int32_t) -> Union[int32_t, uint64_t]
```


### end_i64

```python
def end_i64(code: Union[uint64_t, str], scope: Union[uint64_t, str], table: Union[uint64_t, str]) -> int32_t
```

### remove_i64

```python
def remove_i64(iterator: int32_t) -> None
```

In [23]:
abi = ''
code = '''
import chain
def apply(receiver, code, action):
    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    data = chain.read_action_data()
    if action == name('destroy'):
        while True:
            itr = chain.db_end_i64(code, scope, table)
            if itr == -1:
                break
            itr, primary = chain.db_previous_i64(itr)
            if itr < 0:
                break
            chain.db_remove_i64(itr)
        print('destroy done!')
    elif action == name('initdb'):
        chain.db_store_i64(scope, table, payer, 1, b'1')
        chain.db_store_i64(scope, table, payer, 3, b'3')
        chain.db_store_i64(scope, table, payer, 5, b'5')
        chain.db_store_i64(scope, table, payer, 7, b'7')
        print('init done!')
    elif action == name('test'):
        itr = chain.db_find_i64(code, scope, table, 3)
        data = chain.db_get_i64(itr)
        print('++++get_i64:', data)
        print(itr)
        itr_next, primary = chain.db_next_i64(itr)
        print(itr_next)

        data = chain.db_get_i64(itr_next)

        print('chain.db_next_i64: ', data)
        
        itr_previous, primary = chain.db_previous_i64(itr)
        data = chain.db_get_i64(itr_previous)
        print('chain.db_previous_i64: ', data)
'''

code = uuosapi.mp_compile(python_contract, code)
uuosapi.deploy_python_code(python_contract, code, deploy_type=1)

r = uuosapi.push_action(python_contract, 'destroy', b'', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])

destroy done!



In [26]:
r = uuosapi.push_action(python_contract, 'destroy', b'', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])

r = uuosapi.push_action(python_contract, 'initdb', b'hello,world', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])


destroy done!

522
init done!

712


In [27]:
r = uuosapi.push_action(python_contract, 'test', b'', {python_contract:'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['elapsed'])

++++get_i64: b'3'
1
2
chain.db_next_i64:  b'5'
chain.db_previous_i64:  b'1'

817
